In [12]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np

from tensorflow import keras
from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

In [24]:
with open('pogovorki-plus.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')  # убираем первый невидимый символ

In [25]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('друзья', 8), ('познаются', 3), ('в', 112), ('беде', 3), ('не', 207), ('имей', 4), ('сто', 5), ('рублей', 2), ('а', 89), ('друзей', 8)]


In [26]:
data = tokenizer.texts_to_sequences([texts])
# res = to_categorical(data[0], num_classes=maxWordsCount)
# print(res.shape)
res = np.array( data[0] )

In [27]:
inp_words = 3
n = res.shape[0] - inp_words

X = np.array([res[i:i + inp_words] for i in range(n)])
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount)

In [34]:
model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length = inp_words))
model.add(SimpleRNN(256, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=100)
model.save("model_webp_256_100.h5")

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 3, 256)            256000    
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 1000)              257000    
                                                                 
Total params: 644,328
Trainable params: 644,328
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
102/102 [==============================] - 1s 6ms/step - loss: 6.5164 - accuracy: 0.0569
Epoch 2/100
102/102 [==============================] - 1s 5ms/step - loss: 5.8070 - accuracy: 0.0790
Epoch 3/100
102/102 [==============================] - 1s 5ms/step - loss: 5.1144 - accuracy: 0.1380
Epoch 4/1

102/102 [==============================] - 1s 6ms/step - loss: 0.0720 - accuracy: 0.9622
Epoch 74/100
102/102 [==============================] - 1s 6ms/step - loss: 0.0728 - accuracy: 0.9640
Epoch 75/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0736 - accuracy: 0.9647
Epoch 76/100
102/102 [==============================] - 1s 6ms/step - loss: 0.0721 - accuracy: 0.9619
Epoch 77/100
102/102 [==============================] - 1s 5ms/step - loss: 0.1007 - accuracy: 0.9548
Epoch 78/100
102/102 [==============================] - 1s 6ms/step - loss: 0.1529 - accuracy: 0.9474
Epoch 79/100
102/102 [==============================] - 1s 5ms/step - loss: 0.2193 - accuracy: 0.9309
Epoch 80/100
102/102 [==============================] - 1s 6ms/step - loss: 0.1487 - accuracy: 0.9474
Epoch 81/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0894 - accuracy: 0.9622
Epoch 82/100
102/102 [==============================] - 1s 6ms/step - loss: 0.0826 - accuracy: 

In [29]:
def buildPhrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        # x = to_categorical(data[i: i + inp_words], num_classes=maxWordsCount)  # преобразуем в One-Hot-encoding
        # inp = x.reshape(1, inp_words, maxWordsCount)
        x = data[i: i + inp_words]
        inp = np.expand_dims(x, axis=0)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res

In [39]:
res = buildPhrase("без работы волк")
print(res)

1/1 [==============================] - 0s 27ms/step
без работы волк годом кажется веселишься дважды одно пьёшь делает думай ввечеру что делать поутру без терпенья нет уменья пока ленивый разомнется усердный


In [13]:
model.save("model_web_128_50.h5")